# About data_connector

This notebook pulls information from different sources of data (log, session) to connect student's data across types.
The end goal is a connection between log files and worksheet data.

In [1]:
import os
import numpy as np
import pandas as pd
import getpass
import datetime
import matplotlib
import matplotlib.pyplot as plt
from utils_timeline_viz import find_student_log_file
#utils_read_parsing.py is where all data loading functions live so we set an autoreload on it.
%load_ext autoreload
%autoreload 1
%aimport utils_read_parsing 
from utils_read_parsing import *
%matplotlib inline
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 20, 7
pd.set_option('display.max_columns', None);pd.set_option('display.max_rows', None);pd.set_option('precision', 2)

# Investigating matching ids between log and worksheet data

We load the ids for which we have log files

In [2]:
ids = set(get_students_to_analyze_log())

## beers

We load the metadata for worksheet data

In [3]:
%reload_ext utils_read_parsing
metadata = get_worksheet_metadata('beers')
metadata.head()

,Student ID,Topic,Type,other id,Session,Technical Flags,Comment,use analysis
0,10127163,ABSORBANCE,p,10127163,9,NaN,NaN,True
1,10127163,ABSORBANCE,m,10129163,9,NaN,NaN,True
2,10232160,ABSORBANCE,m,10232160,6-7,NaN,NaN,True
3,10232160,ABSORBANCE,p,10232160,6-7,NaN,factor = detector,True
4,10537160,ABSORBANCE,p,10375160,10,NaN,NaN,True


In [4]:
worksheets = get_worksheet_data_per_sim('beers')
worksheets.head()

,Student ID,Topic,Type,Factors,Model,Correct
0,10127163,ABSORBANCE,p,Concentration,qual,0.0
1,10127163,ABSORBANCE,p,Wavelength,qual,0.0
2,10127163,ABSORBANCE,p,Width,qual,0.0
3,10129163,ABSORBANCE,m,Concentration,qual,1.0
4,10129163,ABSORBANCE,m,Wavelength,qual,0.0


In [5]:
worksheets.groupby(["Type",'Factors','Model'])['Student ID'].nunique()

Type  Factors        Model
m     Concentration  ident    157
                     qual     157
                     quant    157
      Wavelength     ident    157
                     qual     157
                     quant    157
      Width          ident    157
                     qual     157
                     quant    157
p     Concentration  ident    158
                     qual     158
                     quant    158
      Wavelength     ident    158
                     qual     158
                     quant    158
      Width          ident    158
                     qual     158
                     quant    158
Name: Student ID, dtype: int64

In [6]:
ids_pre = set(worksheets[worksheets.Type=='p']['Student ID'])
ids_main = set(worksheets[worksheets.Type=='m']['Student ID'])

For all the ids for which we have log data (ids) we check if we have a pre and a main worksheet data entry. If not in the raw data, than the metadata file will have it corresponding id in it's "other id" column

In [7]:
from tabulate import tabulate
t =[['id with logs','absent from','in metadata other ids', 'other id']]
for i in ids:
    if   i in ids_pre and i in ids_main:
        pass
    elif i in ids_pre and i not in ids_main:
        t.append([i,'main', int(metadata[(metadata['Student ID']==i)&(metadata['Type']=='m')]['Student ID'])==i, int(metadata[(metadata['Student ID']==i)&(metadata['Type']=='m')]['other id'])])
    elif i not in ids_pre and i in ids_main:
        t.append([i,'pre', int(metadata[(metadata['Student ID']==i)&(metadata['Type']=='p')]['Student ID'])==i, int(metadata[(metadata['Student ID']==i)&(metadata['Type']=='p')]['other id'])])
    elif i not in ids_pre and i not in ids_main:
        t.append([i,'both', i in metadata['Student ID'], i in metadata['other id']])
t.sort(key=lambda x: x[1])
print tabulate(t)

------------  -----------  ---------------------  ---------
id with logs  absent from  in metadata other ids  other id
10561164      both         False                  False
10192168      both         False                  False
10908162      main         True                   1090162
11929166      main         True                   119291166
19608154      main         True                   19605154
19989152      main         True                   19984152
15188167      main         True                   15199167
17448165      main         True                   17448105
10127163      main         True                   10129163
13624169      main         True                   13624
16136159      main         True                   16136157
11997159      main         True                   11887159
15298166      pre          True                   5298166
10537160      pre          True                   10375160
14201151      pre          True                   164201151
83145

## caps

Now we do the same for caps as we did for beers

In [8]:
%reload_ext utils_read_parsing
metadata = get_worksheet_metadata('caps')
metadata.head()

,Student ID,Topic,Type,other id,Session,Technical Flags,Comment,use analysis
0,10127163,CAPACITORS,m,10127163,9,NaN,NaN,True
1,10127163,CAPACITORS,p,10127163,9,NaN,NaN,True
2,10232160,CAPACITORS,m,10232160,07-Jun,sp,NaN,True
3,10232160,CAPACITORS,p,10232160,07-Jun,NaN,NaN,True
4,17007153,CAPACITORS,m,1700953,9,sn,NaN,True


In [9]:
worksheets = get_worksheet_data_per_sim('caps')
worksheets.head()

,Student ID,Topic,Type,Factors,Model,Correct
0,10127163,CAPACITORS,m,Area,qual,1.0
1,10127163,CAPACITORS,m,Sep,qual,1.0
2,10127163,CAPACITORS,m,Voltage,qual,1.0
3,10127163,CAPACITORS,p,Area,qual,1.0
4,10127163,CAPACITORS,p,Sep,qual,1.0


In [10]:
worksheets.groupby(["Type",'Factors','Model'])['Student ID'].nunique()

Type  Factors  Model
m     Area     ident    157
               qual     157
               quant    157
      Sep      ident    157
               qual     157
               quant    157
      Voltage  ident    157
               qual     157
               quant    157
p     Area     ident    156
               qual     156
               quant    156
      Sep      ident    156
               qual     156
               quant    156
      Voltage  ident    156
               qual     156
               quant    156
Name: Student ID, dtype: int64

In [11]:
worksheets.groupby(['Student ID'])['Factors'].nunique().head()

Student ID
192168     3
561164     3
1700953    3
3981167    3
4055131    3
Name: Factors, dtype: int64

In [12]:
ids_pre = set(worksheets[worksheets.Type=='p']['Student ID'])
ids_main = set(worksheets[worksheets.Type=='m']['Student ID'])

In [13]:
from tabulate import tabulate
t =[['id with logs','absent from','in metadata other ids', 'other id']]
for i in ids:
    if   i in ids_pre and i in ids_main:
        pass
    elif i in ids_pre and i not in ids_main:
        t.append([i,'main', int(metadata[(metadata['Student ID']==i)&(metadata['Type']=='m')]['Student ID'])==i, int(metadata[(metadata['Student ID']==i)&(metadata['Type']=='m')]['other id'])])
    elif i not in ids_pre and i in ids_main:
        t.append([i,'pre', int(metadata[(metadata['Student ID']==i)&(metadata['Type']=='p')]['Student ID'])==i, int(metadata[(metadata['Student ID']==i)&(metadata['Type']=='p')]['other id'])])
    elif i not in ids_pre and i not in ids_main:
        t.append([i,'both', i in list(metadata['Student ID']), i in list(metadata['other id'])])
t.sort(key=lambda x: x[1])
print tabulate(t)

------------  -----------  ---------------------  --------
id with logs  absent from  in metadata other ids  other id
19780165      both         True                   False
10561164      both         True                   False
10192168      both         True                   False
36107164      both         True                   False
11997159      both         True                   False
23836160      main         True                   23826160
17007153      main         True                   1700953
15444164      main         True                   15444169
10537160      main         True                   10375160
17448165      main         True                   17448105
14201151      main         True                   14201157
90447168      main         True                   80447168
13407169      main         True                   13497169
18922151      pre          True                   18122151
16618163      pre          True                   6618163
14055131      

In [14]:
worksheets.head()

,Student ID,Topic,Type,Factors,Model,Correct
0,10127163,CAPACITORS,m,Area,qual,1.0
1,10127163,CAPACITORS,m,Sep,qual,1.0
2,10127163,CAPACITORS,m,Voltage,qual,1.0
3,10127163,CAPACITORS,p,Area,qual,1.0
4,10127163,CAPACITORS,p,Sep,qual,1.0


## Packaging the worksheet data conveniently

### Fix ids

In [15]:
metadata_L = get_worksheet_metadata('beers')
metadata_C = get_worksheet_metadata('caps')
worksheets_L = get_worksheet_data_per_sim('beers')
worksheets_C = get_worksheet_data_per_sim('caps')

In [28]:
worksheets_L.head()

,Student ID,Topic,Type,Factors,Model,Correct
0,10127163,ABSORBANCE,p,Concentration,qual,0.0
1,10127163,ABSORBANCE,p,Wavelength,qual,0.0
2,10127163,ABSORBANCE,p,Width,qual,0.0
3,10129163,ABSORBANCE,m,Concentration,qual,1.0
4,10129163,ABSORBANCE,m,Wavelength,qual,0.0


In [16]:
#split by sim
pre_L = worksheets_L[worksheets_L.Type=='p']
main_L = worksheets_L[worksheets_L.Type=='m']
pre_C = worksheets_C[worksheets_C.Type=='p']
main_C = worksheets_C[worksheets_C.Type=='m']

#keep only students we actually want to anlyze
pre_L= pre_L[pre_L['Student ID'].isin(metadata_L[(metadata_L['Type']=='p')&metadata_L['Student ID'].isin(ids)]['other id'].values)]
main_L= main_L[main_L['Student ID'].isin(metadata_L[(metadata_L['Type']=='m')&metadata_L['Student ID'].isin(ids)]['other id'].values)]
pre_C= pre_C[pre_C['Student ID'].isin(metadata_C[(metadata_C['Type']=='p')&metadata_C['Student ID'].isin(ids)]['other id'].values)]
main_C= main_C[main_C['Student ID'].isin(metadata_C[(metadata_C['Type']=='m')&metadata_C['Student ID'].isin(ids)]['other id'].values)]


#ids in pre/post match "other id" in worksheet metadata so we need to assign the correct id in logs for each entry in pre/post
pre_L['sid'] = pre_L['Student ID'].apply(lambda row: metadata_L.loc[metadata_L[(metadata_L['other id']==row)].index[0],'Student ID'])
main_L['sid'] = main_L['Student ID'].apply(lambda row: metadata_L.loc[metadata_L[metadata_L['other id']==row].index[0],'Student ID'])
pre_C['sid'] = pre_C['Student ID'].apply(lambda row: metadata_C.loc[metadata_C[(metadata_C['other id']==row)].index[0],'Student ID'])
main_C['sid'] = main_C['Student ID'].apply(lambda row: metadata_C.loc[metadata_C[metadata_C['other id']==row].index[0],'Student ID'])

In [17]:
main_L.head()

,Student ID,Topic,Type,Factors,Model,Correct,sid
3,10129163,ABSORBANCE,m,Concentration,qual,1.0,10127163
4,10129163,ABSORBANCE,m,Wavelength,qual,0.0,10127163
5,10129163,ABSORBANCE,m,Width,qual,1.0,10127163
6,10232160,ABSORBANCE,m,Concentration,qual,0.0,10232160
7,10232160,ABSORBANCE,m,Wavelength,qual,0.0,10232160


If all true we are golden!!!

In [18]:
print set(pre_L['sid'])==ids
print set(pre_C['sid'])==ids
print set(main_L['sid'])==ids
print set(main_C['sid'])==ids

True
True
True
True


### clean up columns

In [19]:
for df,sim in [(pre_L,'L'),(main_L,'L'),(pre_C,'C'),(main_C,'C')]:
    df['sim'] = sim
    df.rename(inplace=True, columns={'Factors':'variable','Model':'model','Correct':'correct','Type':'worksheet'})
    df.replace('Voltage','Battery voltage',inplace=True)
    df.replace('Sep','Separation',inplace=True)
    df.drop(['Student ID','Topic'],inplace=True,axis=1)

In [20]:
pre_L.head()

,worksheet,variable,model,correct,sid,sim
0,p,Concentration,qual,0.0,10127163,L
1,p,Wavelength,qual,0.0,10127163,L
2,p,Width,qual,0.0,10127163,L
9,p,Concentration,qual,NaN,10232160,L
10,p,Wavelength,qual,NaN,10232160,L


### "unmelt" data using pivot table

Add a column for presence of absence of a type of model statement for each ident, qual, quant

In [21]:
for df,worksheet_type in [(pre_L,'pre'),(main_L,'main'),(pre_C,'pre'),(main_C,'main')]:
    df[worksheet_type + '_statement'] = df['correct'].copy()
    df[worksheet_type + '_statement'].replace(0,1, inplace = True)
    df[worksheet_type + '_statement'].replace(1,1, inplace = True)
    df[worksheet_type + '_statement'].fillna(0, inplace = True)
    df.rename(columns={'correct':worksheet_type +'_correct'}, inplace = True)

In [22]:
main_L.head(12)

,worksheet,variable,model,main_correct,sid,sim,main_statement
3,m,Concentration,qual,1.0,10127163,L,1.0
4,m,Wavelength,qual,0.0,10127163,L,1.0
5,m,Width,qual,1.0,10127163,L,1.0
6,m,Concentration,qual,0.0,10232160,L,1.0
7,m,Wavelength,qual,0.0,10232160,L,1.0
8,m,Width,qual,0.0,10232160,L,1.0
15,m,Concentration,qual,1.0,10375163,L,1.0
16,m,Wavelength,qual,NaN,10375163,L,0.0
17,m,Width,qual,1.0,10375163,L,1.0
21,m,Concentration,qual,NaN,10420167,L,0.0


We can concatenate both pre and posts per sim

In [23]:
pre_C.shape, pre_L.shape

((1332, 7), (1332, 7))

In [24]:
main = pd.concat([main_L,main_C],axis=0)
print main.shape
pre = pd.concat([pre_L,pre_C],axis=0)
print pre.shape
pre.sort_values(["sid",'model']).head(24)

(2664, 7)
(2664, 7)


,worksheet,variable,model,pre_correct,sid,sim,pre_statement
861,p,Concentration,ident,NaN,10127163,L,0.0
862,p,Wavelength,ident,NaN,10127163,L,0.0
863,p,Width,ident,NaN,10127163,L,0.0
858,p,Area,ident,NaN,10127163,C,0.0
859,p,Separation,ident,NaN,10127163,C,0.0
860,p,Battery voltage,ident,NaN,10127163,C,0.0
0,p,Concentration,qual,0.0,10127163,L,1.0
1,p,Wavelength,qual,0.0,10127163,L,1.0
2,p,Width,qual,0.0,10127163,L,1.0
3,p,Area,qual,1.0,10127163,C,1.0


We merge pre and post based on sid, variable, model (!!!) and check the integrity or data

In [25]:
data = pre.drop('worksheet',axis=1).merge(main.drop(['sim','worksheet'],axis=1), how='inner',on = ['sid','variable','model'])
data.sort_values(['sid','variable','model'],inplace=True)
data = data[['sid','sim','variable','model','pre_statement','pre_correct','main_statement','main_correct']]
data.reset_index(inplace=True,drop=True);

In [26]:
data.head()

,sid,sim,variable,model,pre_statement,pre_correct,main_statement,main_correct
0,10127163,C,Area,ident,0.0,NaN,0.0,NaN
1,10127163,C,Area,qual,1.0,1.0,1.0,1.0
2,10127163,C,Area,quant,1.0,1.0,1.0,1.0
3,10127163,C,Battery voltage,ident,0.0,NaN,0.0,NaN
4,10127163,C,Battery voltage,qual,1.0,1.0,1.0,1.0


In [27]:
data.to_csv(os.path.join(BIG_FOLDER,'all_massaged_data\\worksheets_models.txt'), sep='\t', index=False)